In [1]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../adult-data"        

In [2]:
# (fold cell) Load and prep web browsing data
# output = df with visits to adult sites
usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols = ["caseid", "category", "private_domain", "page_duration", "session_start_time"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
         # Renaming columns to be consistent w/ web_mobile & web_desktop
         .rename_column("session_start_time", "visit_time_local")
         .rename_column("page_duration", "visit_duration")
        )
    ])
     )
print("output = df with web browses")
print(f"{len(df)=:,}")
df.head()

output = df with web browses
len(df)=6,297,382


,caseid,private_domain,category,visit_time_local,visit_duration
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457
2,205323077,google.com,Business,2022-06-01 01:15:12,55
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10


In [3]:
# (fold cell) Load the 1200 individual metadata
df_ind_metadata = (
    pd.read_csv(os.path.join(DATAPATH, "SOOD0001_OUTPUT.csv"))
    .case_when(
        lambda df: df.pid7==1, "D", 
        lambda df: df.pid7==2, "D", 
        lambda df: df.pid7==3, "D", 
        lambda df: df.pid7==4, "I", 
        lambda df: df.pid7==5, "R", 
        lambda df: df.pid7==6, "R",  
        lambda df: df.pid7==7, "R", 
        np.nan, 
        column_name="party"
    )
    .assign(
        caseid=lambda df: df.caseid.apply(int),
        birthyr=lambda df: df.birthyr.apply(int),
    )
)
print(f"{len(df_ind_metadata)=}")
df_ind_metadata.head(3)

len(df_ind_metadata)=1200


/home/lsys/miniconda3/lib/python3.10/site-packages/pandas_flavor/register.py:153: DeprecationWarning: The last argument in the variable arguments has been assigned as the default. Note however that this will be deprecated in a future release; use an even number of boolean conditions and values, and pass the default argument to the `default` parameter instead.
  return method(self._obj, *args, **kwargs)


,caseid,birthyr,gender,race,educ,pid3,pid7,presvote20post,inputstate,region,party
0,200661421,1963,2,1,4,3,3,1,39,2,D
1,200686597,1992,2,6,5,5,8,-1,48,3,NaN
2,200953869,1959,2,1,5,2,7,2,42,1,R


In [4]:
# Read in list of classified FP (false positives)
falsepositives = (pd
                  .read_csv("../data/false_positives_list.csv")
                  ["flagged as FP"].tolist()
                 )
falsepositives

['9gag.com',
 'sniffies.com',
 'f95zone.to',
 'silverdaddies.com',
 'ttcache.com',
 'xlviirdr.com',
 '4chan.org',
 'thechive.com',
 'victoriassecret.com',
 'usasexguide.nl',
 'xlviiirdr.com',
 'xxxjmp.com',
 'urbandictionary.com',
 'lpsg.com']

In [5]:
# (fold cell) Get individual level total visit durations to adult and non-adult sites
df_ind = (
    df_ind_metadata
    # Get total adult site visits
    .merge(
        (
            df
            .query("category.str.contains('adult', case=False, na=False)")
            .query("private_domain not in @falsepositives")
            .groupby("caseid")
            .size()
            .reset_index()
            .rename_column(0, "visits_adult")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # Get total non-adult site visits
    .merge(
        (
            df
            .query("~category.str.contains('adult', case=False, na=False)|private_domain in @falsepositives")
            .groupby("caseid")
            .size()
            .reset_index()
            .rename_column(0, "visits_nonadult")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # If adult or non-adult (exclusive) is nan, impute as zero
    .assign(
        visits_adult=lambda df: np.where(
            pd.isna(df.visits_adult) & ~pd.isna(df.visits_nonadult), 0, df.visits_adult
        )
    )
    .assign(
        visits_nonadult=lambda df: np.where(
            ~pd.isna(df.visits_adult) & pd.isna(df.visits_nonadult),
            0,
            df.visits_nonadult,
        )
    )  # If adult or non-adult (exclusive) is nan, impute as zero
    .assign(
        visits_adult=lambda df: np.where(
            pd.isna(df.visits_adult) & ~pd.isna(df.visits_nonadult), 0, df.visits_adult
        )
    )
    .assign(
        visits_nonadult=lambda df: np.where(
            ~pd.isna(df.visits_adult) & pd.isna(df.visits_nonadult),
            0,
            df.visits_nonadult,
        )
    )
    # Get total visits
    .assign(
        visits=lambda df: df.visits_adult + df.visits_nonadult,
        prop_adult_visits=lambda df: 100 * (df.visits_adult / df.visits),
    )
    .assign(rep=lambda df: np.where(df.party == "R", 1, 0))
    # Get adult visits duration
    .merge(
        (
            df
            .query("category.str.contains('adult', case=False, na=False)")
            .query("private_domain not in @falsepositives")
            .groupby("caseid")["visit_duration"]
            .sum()
            .reset_index()
            .rename_column("visit_duration", "duration_adult")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # Get non-adult visits duration
    .merge(
        (
            df
            .query("~category.str.contains('adult', case=False, na=False)|private_domain in @falsepositives")
            .groupby("caseid")["visit_duration"]
            .sum()
            .reset_index()
            .rename_column("visit_duration", "duration_nonadult")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # If adult or non-adult (exclusive) duration is nan, impute as zero
    .assign(
        duration_adult=lambda df: np.where(
            pd.isna(df.duration_adult) & ~pd.isna(df.duration_nonadult),
            0,
            df.duration_adult,
        )
    )
    .assign(
        duration_nonadult=lambda df: np.where(
            ~pd.isna(df.duration_adult) & pd.isna(df.duration_nonadult),
            0,
            df.duration_nonadult,
        )
    )
    # Get total duration
    .assign(
        duration=lambda df: df.duration_adult + df.duration_nonadult,
        prop_adult_duration=lambda df: 100 * (df.duration_adult / df.duration),
    )        
    # Define rep dummy
    .case_when(
        lambda df: df.party == "R",
        1,
        lambda df: df.party == "D",
        0,
        np.nan,
        column_name="rep",
    )
    # Define dummy for visiting adultsites
    .case_when(
        lambda df: df.visits_adult > 0,
        1,
        lambda df: df.visits_adult == 0,
        0,
        np.nan,
        column_name="yes_visit_adults",
    )
    # Collapse race into 5 groups
    .case_when(lambda df: df.race >= 5, 5, lambda df: df.race, column_name="race2")
    # Collapse birthyr into cohorts (e.g. 1960s, 1970s,)
    .assign(cohort=lambda df: df.birthyr.round(decimals=-1))
    # Age
    .assign(age=lambda df: 2022 - df.birthyr + 1)
    .assign(age2=lambda df: df.age**2)
    # Collapse educ to 4 groups
    .case_when(
        lambda df: df.educ == 1,
        1,  # did not grad HS
        lambda df: df.educ == 2,
        2,  # HS
        lambda df: df.educ == 3,
        3,  # some college
        lambda df: df.educ >= 4,
        4,  # College grad
        np.nan,
        column_name="educ2",
    )
    .assign(educ2=lambda df: df.educ2.apply(int))
)
assert df_ind.prop_adult_visits.min() >= 0
assert df_ind.prop_adult_visits.max() <= 100
df_ind

/home/lsys/miniconda3/lib/python3.10/site-packages/pandas_flavor/register.py:153: DeprecationWarning: The last argument in the variable arguments has been assigned as the default. Note however that this will be deprecated in a future release; use an even number of boolean conditions and values, and pass the default argument to the `default` parameter instead.
  return method(self._obj, *args, **kwargs)
/home/lsys/miniconda3/lib/python3.10/site-packages/pandas_flavor/register.py:153: DeprecationWarning: The last argument in the variable arguments has been assigned as the default. Note however that this will be deprecated in a future release; use an even number of boolean conditions and values, and pass the default argument to the `default` parameter instead.
  return method(self._obj, *args, **kwargs)
/home/lsys/miniconda3/lib/python3.10/site-packages/pandas_flavor/register.py:153: DeprecationWarning: The last argument in the variable arguments has been assigned as the default. Note how

,caseid,birthyr,gender,race,educ,pid3,pid7,presvote20post,inputstate,region,party,visits_adult,visits_nonadult,visits,prop_adult_visits,rep,duration_adult,duration_nonadult,duration,prop_adult_duration,yes_visit_adults,race2,cohort,age,age2,educ2
0,200661421,1963,2,1,4,3,3,1,39,2,D,0.0,1728.0,1728.0,0.000000,0.0,0.0,77642.0,77642.0,0.000000,0.0,1,1960,60,3600,4
1,200686597,1992,2,6,5,5,8,-1,48,3,NaN,243.0,3994.0,4237.0,5.735190,NaN,25952.0,234196.0,260148.0,9.975860,1.0,5,1990,31,961,4
2,200953869,1959,2,1,5,2,7,2,42,1,R,0.0,20.0,20.0,0.000000,1.0,0.0,1182.0,1182.0,0.000000,0.0,1,1960,64,4096,4
3,201302005,1966,2,2,3,5,8,1,12,3,NaN,20.0,527.0,547.0,3.656307,NaN,6166.0,23855.0,30021.0,20.538956,1.0,2,1970,57,3249,3
4,201590505,1977,1,4,5,3,3,1,6,4,D,0.0,11086.0,11086.0,0.000000,0.0,0.0,383161.0,383161.0,0.000000,0.0,4,1980,46,2116,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1802885,1935,1,1,6,4,5,2,12,3,R,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,1940,88,7744,4
1196,1809723,1943,1,1,2,1,1,1,51,3,D,0.0,9805.0,9805.0,0.000000,0.0,0.0,1272613.0,1272613.0,0.000000,0.0,1,1940,80,6400,2
1197,1827351,1947,1,1,5,3,4,1,31,2,I,0.0,543.0,543.0,0.000000,NaN,0.0,23929.0,23929.0,0.000000,0.0,1,1950,76,5776,4
1198,1924249,1980,1,7,4,2,7,2,6,4,R,182.0,21379.0,21561.0,0.844117,1.0,2094.0,309876.0,311970.0,0.671218,1.0,5,1980,43,1849,4


In [6]:
df_ind.to_csv("../data/individual_browsing_data.csv", index=False)